In [1]:
from collections import Counter


def drop_datetime(txt: str) -> str:
    """Strip away the datetime information of each message."""
    
    txt = txt.split("\n")
    txt_clean = [i[21:] for i in txt]
    txt_joined = "\n".join(txt_clean)
    return txt_joined


def get_infrequent_chars(txt: str, min_count: int) -> list:
    chars_counts = Counter(text)
    chars_remove = [k for k,v in chars_counts.items() if v< min_count]
    return chars_remove


def drop_chars(txt: str, drop: list) -> str:
    return txt.translate(str.maketrans("", "", "".join(drop)))


In [2]:
text = open("assets/chat.txt", "r").read()
text_0 = drop_datetime(text)

infreq_chars = get_infrequent_chars(text_0, min_count=200)
text_1 = drop_chars(text_0, infreq_chars)

In [4]:
import torch


def get_vocab(text: str) -> list:
    """Returns a sorted list of all available characters in the corpus."""

    return sorted(list(set("".join(text))))


def encode(s: str, vocab: list) -> torch.tensor:
    """Encode a string into a tensor of integers, given a fixed vocabulary."""

    map = {s:i for i,s in enumerate(vocab)}
    enc = [map[c] for c in s]
    enc = torch.tensor(enc, dtype=torch.long)
    return enc

In [6]:
vocab = get_vocab(text_1)
vocab_size = len(vocab)

In [8]:
print(vocab_size)

139
